In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
df = pd.read_csv("insurancedata.csv")
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size = 0.2,random_state = 25)

In [ ]:
len(X_train)

In [ ]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] /100
X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] /100

In [ ]:
X_train_scaled

In [ ]:
# Building neural network
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])
model.compile(optimizer ='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])
# EARLY STOPPING
es = EarlyStopping(
    monitor='val_loss',
    patience=200,
    verbose = 1,
    mode=min
)
# FIT THE MODEL

history = model.fit(X_train_scaled,y_train,epochs=5000,callbacks=[es])

In [ ]:
model.evaluate(X_test_scaled,y_test)

In [ ]:
coef , intercept = model.get_weights()
coef , intercept

In [ ]:
model.save("model.h5")

In [ ]:
# RAW METHOD WITHOUT TENSORFLOW
def sigmoid(x):
    import math
    return (1/ 1 + math.exp(-x))
sigmoid(18)

In [ ]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [ ]:
prediction_function(.47,1)

In [ ]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)*(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
def sigmoid_numpy(x):
    return (1/1+np.exp(-x))
sigmoid_numpy(np.array([12,0,1]))

In [ ]:
def gradient_descent( age, affordibility , y_true , epochs, loss_threshold):
    # w1 , w2 ,bias 
    w1 = w2 = 1
    bias = 0
    learning_rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility+bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true , y_predicted)
        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))
        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - learning_rate*w1d
        w2 = w2 - learning_rate*w2d
        bias = bias - learning_rate * bias_d
        print(f'epoch:{i}, w1:{w1},w2:{w2},bias:{bias},loss:{loss}')
        return w1 , w2 , bias

In [ ]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4631)

In [ ]:
coef,intercept

In [ ]:
model.save("model.keras")

In [ ]:
model.export('model')

In [ ]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
    def fit(self,X,y,epochs,loss_threshold):
        self.w1,self.w2,self.bias = self.gradient_descent(X['age'],X['affordibility'],y,epochs , loss_threshold)
    def gradient_descent( age, affordibility , y_true , epochs, loss_threshold):
    # w1 , w2 ,bias 
        w1 = w2 = 1
        bias = 0
        learning_rate = 0.5
        n = len(age)
        for i in range(epochs):
             weighted_sum = w1*age + w2*affordibility+bias
             y_predicted = sigmoid_numpy(weighted_sum)
             loss = log_loss(y_true , y_predicted)
             w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
             w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))
             bias_d = np.mean(y_predicted-y_true)
             w1 = w1 - learning_rate*w1d
             w2 = w2 - learning_rate*w2d
             bias = bias - learning_rate * bias_d
             print(f'epoch:{i}, w1:{w1},w2:{w2},bias:{bias},loss:{loss}')
      
         if loss<= loss_threshold
             break
    return w1,w2, bias
    
        

In [ ]:
custommodel = myNN()
custommodel.fit(X_train_scaled,y_train,epochs = 10,loss_threshold = 0.4631)